In [26]:
import gradio as gr
from openai import OpenAI
import base64
import time

In [27]:
client = OpenAI(
  base_url="", # USE vllm api-server address or URL,
  api_key= "EMPTY",
)

In [ ]:
models = client.models.list()
model = models.data[0].id

In [ ]:
def encode_base64_content_from_file(image_path: str) -> str:
    """Encode a content retrieved from image_path to base64 format."""
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

In [64]:
def add_message(history, message):
    for x in message["files"]:
        history.append({"role": "user", "content": {"path": x}})
    if message["text"] is not None:
        history.append({"role": "user", "content": message["text"]})
    return history, gr.MultimodalTextbox(value=None, interactive=False)

In [68]:
def bot(history: list):
    response = "실제 vllm output입니다."
    history.append({"role": "assistant", "content": ""})
    for character in response:
        history[-1]["content"] += character
        time.sleep(0.05)
        yield history

In [66]:
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(elem_id = "chatbot", bubble_full_width=False, type="messages")

  chat_input = gr.MultimodalTextbox(
    interactive=True,
    file_count="multiple",
    placeholder="메세지나 파일을 입력하세요",
    show_label=False,
  )

  chat_msg = chat_input.submit(
    add_message, [chatbot, chat_input], [chatbot,chat_input]
  )
  bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
  bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

In [67]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
